# Persistence model

Here is an example of how to run the persistence model from a notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools
import hydra
import torch
import torch.nn as nn

In [ ]:
from smc01.postprocessing.dataset import SMCTimeSeriesDataset
from smc01.postprocessing.transform import CompositeTransform, DataframeToXarray, TimeseriesXarrayToExample

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('validate', ['experiment=persistence_gdps', 'experiment.dataset.begin=2021-05-01', 'experiment.dataset.end=2021-10-01'])

In [ ]:
dataset = hydra.utils.instantiate(cfg.experiment.dataset)

In [ ]:
n_stations = len(dataset.stations)

In [ ]:
sample = dataset[0]
n_features = dataset[0]['features'].shape[-1]

In [ ]:
m = hydra.utils.instantiate(cfg.experiment.model, n_stations, n_features)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, num_workers=4)

In [ ]:
for b in itertools.islice(loader, 10):
    prediction = m(b)
    error = torch.square(b['obs'] - prediction)
    
    print(torch.sqrt(error.mean()))